In [1]:
!pip install hyperopt

In [9]:
import pandas as pd
import numpy as np
import os
import datetime

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [3]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/data'

/content/drive/My Drive/Colab Notebooks/dw_matrix/data


In [0]:
labels = pd.read_csv('sign_labels.csv')
labels_dict = labels.to_dict()['b']

In [0]:
train = pd.read_pickle('sign_train.p')
test = pd.read_pickle('sign_test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1:  y_train = to_categorical(y_train)
if y_test.ndim == 1:  y_test = to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def get_model(input_shape, num_classes):
  model = Sequential([
                      Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
                      Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
                      MaxPool2D(),
                      Dropout(0.3),

                      Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
                      Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
                      MaxPool2D(),
                      Dropout(0.3),

                      Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
                      Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
                      MaxPool2D(),
                      Dropout(0.3),                      

                      Flatten(),

                      Dense(1024, activation='relu'),
                      Dropout(0.3),

                      Dense(1024, activation='relu'),
                      Dropout(0.3),
                      
                      Dense(num_classes, activation='softmax')                
  ])

  return model

In [0]:
def get_model(params):
  model = Sequential([
                      Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
                      Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
                      MaxPool2D(),
                      Dropout(params['dropout_cnn_1']),

                      Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
                      Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
                      MaxPool2D(),
                      Dropout(params['dropout_cnn_2']),

                      Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
                      Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
                      MaxPool2D(),
                      Dropout(params['dropout_cnn_2']),                      

                      Flatten(),

                      Dense(1024, activation='relu'),
                      Dropout(params['dropout_dense_1']),

                      Dense(1024, activation='relu'),
                      Dropout(params['dropout_dense_2']),
                      
                      Dense(num_classes, activation='softmax')                
  ])

  return model

In [0]:
def f_object(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  
  model.fit(
      X_train, 
      y_train,
      batch_size = int(params.get('batch_size', 128)),
      epochs = 5,
      verbose = 0
      )
  
  score = model.evaluate(X_test, y_test, verbose = 0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [26]:
space = {'batch_size': hp.quniform('batch_size', 100, 200, 10),
         'dropout_cnn_1': hp.uniform('dropout_cnn_1', 0.2, 0.5),
         'dropout_cnn_1': hp.uniform('dropout_cnn_1', 0.2, 0.5),
         'dropout_cnn_2': hp.uniform('dropout_cnn_2', 0.2, 0.5),
         'dropout_cnn_3': hp.uniform('dropout_cnn_3', 0.2, 0.5),
         'dropout_dense_1': hp.uniform('dropout_dense_1', 0.3, 0.7),
         'dropout_dense_2': hp.uniform('dropout_dense_2', 0.3, 0.7),


}

best = fmin(f_object,
            space,
            tpe.suggest,
            30,
            Trials())

{'batch_size': 170.0, 'dropout_cnn_1': 0.3522436145803961, 'dropout_cnn_2': 0.4371045883003841, 'dropout_cnn_3': 0.3496312013433581, 'dropout_dense_1': 0.6965453480456396, 'dropout_dense_2': 0.49076345288095236}
accuracy=0.8519274592399597
{'batch_size': 180.0, 'dropout_cnn_1': 0.20132649456435733, 'dropout_cnn_2': 0.3742164610141008, 'dropout_cnn_3': 0.3315588534749892, 'dropout_dense_1': 0.3918465364592121, 'dropout_dense_2': 0.3651742311803727}
accuracy=0.9721088409423828
{'batch_size': 130.0, 'dropout_cnn_1': 0.291495775739288, 'dropout_cnn_2': 0.4359668373348271, 'dropout_cnn_3': 0.32025563030632664, 'dropout_dense_1': 0.5113918651314687, 'dropout_dense_2': 0.4542586126379185}
accuracy=0.920181393623352
{'batch_size': 180.0, 'dropout_cnn_1': 0.2827479396048895, 'dropout_cnn_2': 0.3125104319526302, 'dropout_cnn_3': 0.4445390298513405, 'dropout_dense_1': 0.6207996811543826, 'dropout_dense_2': 0.4685238109904858}
accuracy=0.9541950225830078
{'batch_size': 180.0, 'dropout_cnn_1': 0.28

In [0]:
!git config --global user.email "m.zajac1988@gmail.com"
!git config --global user.name "Mateusz"

In [0]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three'

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three


In [0]:
!git add day4.ipynb

In [0]:
!git commit -m 'Train model on grayscale road signs'

[master 4b87476] Train model on grayscale road signs
 1 file changed, 1 insertion(+), 1 deletion(-)


In [0]:
!git push -u origin master

Counting objects: 8, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (8/8), done.
Writing objects: 100% (8/8), 3.89 KiB | 1.95 MiB/s, done.
Total 8 (delta 5), reused 0 (delta 0)
remote: Resolving deltas: 100% (5/5), completed with 2 local objects.
To https://github.com/mattzajac/dw_matrix.git
   1f2b1db..4b87476  master -> master
Branch 'master' set up to track remote branch 'master' from 'origin'.
